In [1]:
import json
from multiprocessing import Process, JoinableQueue
import argparse
import os
import re
import shutil
import sys
import glob
import numpy as np
from unicodedata import normalize
from skimage import io
from skimage.color import rgb2hsv
from skimage.util import img_as_ubyte
from skimage import filters
from PIL import Image, ImageFilter, ImageStat
import math



In [2]:

Image.MAX_IMAGE_PIXELS = None

if os.name == 'nt':
	_dll_path = os.getenv('OPENSLIDE_PATH')
	if _dll_path is not None:
		if hasattr(os, 'add_dll_directory'):
			# Python >= 3.8
			with os.add_dll_directory(_dll_path):
				import openslide
		else:
			# Python < 3.8
			_orig_path = os.environ.get('PATH', '')
			os.environ['PATH'] = _orig_path + ';' + _dll_path
			import openslide
			os.environ['PATH'] = _orig_path
else:
	import openslide
from openslide import open_slide, ImageSlide
from openslide.deepzoom import DeepZoomGenerator


In [3]:
c_slide = '/data10/shared/tcga_all/brca/TCGA-A1-A0SF-01Z-00-DX1.7F252D89-EA78-419F-A969-1B7313D77499.svs'
temp_slide = open_slide(c_slide)

base_level_mag = int(temp_slide.properties['aperio.AppMag'][:2])
base_level_mag

40

In [4]:
!cp '/data10/shared/tcga_all/brca/TCGA-A1-A0SF-01Z-00-DX1.7F252D89-EA78-419F-A969-1B7313D77499.svs' '/data04/shared/skapse/Interpretable MIL/Datasets/test_dataset/slides/'


In [5]:
!cp '/data04/shared/skapse/Cell_guided/Data/TCGA_BRCA/Hovernet_output/json/TCGA-A1-A0SF-01Z-00-DX1.7F252D89-EA78-419F-A969-1B7313D77499.json' '/data04/shared/skapse/Interpretable MIL/Datasets/test_dataset/Hovernet_output/json/'


In [6]:
!cp '/data04/shared/skapse/Interpretable MIL/Datasets/TCGA_BRCA/cell_property/TCGA-A1-A0SF-01Z-00-DX1.7F252D89-EA78-419F-A969-1B7313D77499.pickle' '/data04/shared/skapse/Interpretable MIL/Datasets/test_dataset/cell_property/'


In [2]:
import os
from tqdm import tqdm
import pickle
import numpy as np
import pandas as pd
import argparse


In [ ]:
--feat_path '/data04/shared/skapse/Interpretable MIL/Datasets/test_dataset/Handcrafted_features' 
--list_dict_path '/data04/shared/skapse/Interpretable MIL/Datasets/test_dataset/patches' 
--column_name_path '/data04/shared/skapse/Interpretable MIL/Datasets/test_dataset'

In [5]:
feat_path = '/data04/shared/skapse/Interpretable MIL/Datasets/test_dataset/Handcrafted_features' 
column_name_path = '/data04/shared/skapse/Interpretable MIL/Datasets/test_dataset'
list_dict_path = '/data04/shared/skapse/Interpretable MIL/Datasets/test_dataset/patches' 

with open(os.path.join(list_dict_path, 'all_dict.pickle'), 'rb') as f:
	all_dict = pickle.load(f)

with open(os.path.join(column_name_path,'cell_stat_column_name.pickle'), 'rb') as f:
	cell_stat_column_name = pickle.load(f)
	
with open(os.path.join(column_name_path,'sna_column_name.pickle'), 'rb') as f:
	sna_column_name = pickle.load(f)
	
with open(os.path.join(column_name_path,'athena_column_name.pickle'), 'rb') as f:
	athena_column_name = pickle.load(f)

	
athena_dir = feat_path + '/athena_statistics'
cell_dir = feat_path + '/cell_statistics'
sna_dir = feat_path + '/sna_statistics'


all_dict_key_list = []

for i, _ in enumerate(list(all_dict.keys())):
	exist = False
	for j in os.listdir(cell_dir):
		if _ + '.' in j:
			exist = True
			break
	if exist:
		all_dict_key_list.append(j)

print(len(all_dict_key_list), len(all_dict))

select_dir = cell_dir

feat_list = []

for i in tqdm(all_dict_key_list):
	with open(select_dir + '/' + i, 'rb') as f:
		temp_feat = pickle.load(f)
		
	feat_list.append(temp_feat)

merged_dict = {}

for d in tqdm(feat_list):
	merged_dict.update(d)
	
df_cell = pd.DataFrame.from_dict(merged_dict, orient='index', columns=cell_stat_column_name)

	

select_dir = sna_dir

feat_list = []

for i in tqdm(all_dict_key_list):
	with open(select_dir + '/' + i, 'rb') as f:
		temp_feat = pickle.load(f)
		
	feat_list.append(temp_feat)
		
merged_dict = {}

for d in tqdm(feat_list):
	merged_dict.update(d)
		
df_sna = pd.DataFrame.from_dict(merged_dict, orient='index', columns=sna_column_name)

	

select_dir = athena_dir
feat_list = []

for i in tqdm(all_dict_key_list):
	with open(select_dir + '/' + i, 'rb') as f:
		temp_feat = pickle.load(f)
		
	feat_list.append(temp_feat)
		
merged_dict = {}

for d in tqdm(feat_list):
	merged_dict.update(d)
	  
df_athena = pd.DataFrame.from_dict(merged_dict, orient='index', columns=athena_column_name)


features_csv = pd.concat((df_cell, df_sna, df_athena), axis=1)

1 1


100%|███████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 4629.47it/s]


In [4]:
df_merged

,number of neoplastic cells,number of inflamatory cells,number of connective cells,number of necrosis cells,number of no-neoplastic cells,neoplastic cells: mean of their area,neoplastic cells: mean of their perimeter,neoplastic cells: mean of their orientation,neoplastic cells: mean of their eccentricity,neoplastic cells: mean of their solidity,...,Local Renyi entropy (at alpha=infinity) histogram-bin: 3,Local Renyi entropy (at alpha=infinity) histogram-bin: 4,Infiltration of inflamatory cells in neoplastic region,Infiltration of connective cells in neoplastic region,Infiltration of necrosis cells in neoplastic region,Infiltration of no-neoplastic cells in neoplastic region,Infiltration of neoplastic cells in inflamatory region,Infiltration of neoplastic cells in connective region,Infiltration of neoplastic cells in necrosis region,Infiltration of neoplastic cells in no-neoplastic region
/data04/shared/skapse/Interpretable MIL/Datasets/test_dataset/patches/TCGA-A1-A0SF-01Z-00-DX1/14_32_2.jpg,119.0,247.0,227.0,124.0,11.0,600.568807,92.948486,-0.008262,0.694020,0.958086,...,19,0,0.149254,0.708955,0.119403,0.108209,0.075188,0.512129,0.169312,5.800000
/data04/shared/skapse/Interpretable MIL/Datasets/test_dataset/patches/TCGA-A1-A0SF-01Z-00-DX1/14_30_2.jpg,120.0,8.0,148.0,25.0,9.0,611.620370,93.246167,0.051500,0.696937,0.958325,...,6,0,0.030075,0.969925,0.109023,0.109023,NaN,0.798762,1.160000,4.833333
/data04/shared/skapse/Interpretable MIL/Datasets/test_dataset/patches/TCGA-A1-A0SF-01Z-00-DX1/14_33_2.jpg,406.0,135.0,380.0,82.0,30.0,569.128415,90.400441,0.051082,0.695777,0.960034,...,37,1,0.071496,0.501411,0.038570,0.098777,0.415301,0.714477,0.471264,5.000000
/data04/shared/skapse/Interpretable MIL/Datasets/test_dataset/patches/TCGA-A1-A0SF-01Z-00-DX1/14_29_2.jpg,150.0,12.0,254.0,9.0,27.0,736.867647,102.905439,0.037435,0.670672,0.960748,...,8,0,0.069307,1.339934,0.036304,0.165017,7.000000,0.669967,2.750000,2.083333
/data04/shared/skapse/Interpretable MIL/Datasets/test_dataset/patches/TCGA-A1-A0SF-01Z-00-DX1/14_28_2.jpg,21.0,4.0,43.0,23.0,1.0,1009.578947,120.927292,-0.394998,0.697753,0.964670,...,2,0,0.062500,2.468750,0.312500,0.062500,1.000000,0.951807,0.238095,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
/data04/shared/skapse/Interpretable MIL/Datasets/test_dataset/patches/TCGA-A1-A0SF-01Z-00-DX1/14_22_43.jpg,0.0,5.0,27.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN
/data04/shared/skapse/Interpretable MIL/Datasets/test_dataset/patches/TCGA-A1-A0SF-01Z-00-DX1/14_23_43.jpg,0.0,4.0,36.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
/data04/shared/skapse/Interpretable MIL/Datasets/test_dataset/patches/TCGA-A1-A0SF-01Z-00-DX1/14_24_43.jpg,11.0,20.0,26.0,0.0,2.0,474.545455,80.964292,0.087753,0.646105,0.950707,...,3,0,2.500000,2.416667,0.000000,0.333333,1.000000,0.500000,NaN,NaN
/data04/shared/skapse/Interpretable MIL/Datasets/test_dataset/patches/TCGA-A1-A0SF-01Z-00-DX1/14_23_44.jpg,0.0,3.0,16.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN


In [6]:
use_kfunction = 'False'
# remove_cell_type = 'no-neoplastic'
remove_cell_type = 'none'
norm_feat = 'bin'
bins = 10
divide = 1

print(features_csv.shape)


(825, 387)


In [7]:

if use_kfunction == 'False':
	include_list = []
	for i, _ in enumerate(list(features_csv.columns)):
		if 'inflamatory cells: k-function' in _:
			continue

		if 'connective cells: k-function' in _:
			continue

		if 'necrosis cells: k-function' in _:
			continue

		if 'no-neoplastic cells: k-function' in _:
			continue

		include_list.append(i)

	features_csv = features_csv.iloc[:, np.array(include_list)]

	print('After removing k-function of other cells', features_csv.shape)

print('remove_cell_type:', remove_cell_type)

include_list = []
for i, _ in enumerate(list(features_csv.columns)):
	if not remove_cell_type in _:
		include_list.append(i)

features_csv = features_csv.iloc[:, np.array(include_list)]
feats_size = len(include_list)

print('After removing cell type ' + remove_cell_type + ' columns', features_csv.shape)
print('Feat size after:', feats_size)


After removing k-function of other cells (825, 367)
remove_cell_type: none
After removing cell type none columns (825, 367)
Feat size after: 367


In [8]:

shannon_group = ['Local Shannon index histogram-bin: 0', 'Local Shannon index histogram-bin: 1',  'Local Shannon index histogram-bin: 2', 'Local Shannon index histogram-bin: 3', 'Local Shannon index histogram-bin: 4']

simpson_group = ['Local Simpson index histogram-bin: 0', 'Local Simpson index histogram-bin: 1',  'Local Simpson index histogram-bin: 2', 'Local Simpson index histogram-bin: 3', 'Local Simpson index histogram-bin: 4']

renqi_group = ['Local Renyi entropy (at alpha=infinity) histogram-bin: 0', 'Local Renyi entropy (at alpha=infinity) histogram-bin: 1',  'Local Renyi entropy (at alpha=infinity) histogram-bin: 2', 'Local Renyi entropy (at alpha=infinity) histogram-bin: 3', 'Local Renyi entropy (at alpha=infinity) histogram-bin: 4']


richness_group = ['Local Richness, number of times 1 cell-types present', 'Local Richness, number of times 2 cell-types present',	'Local Richness, number of times 3 cell-types present',	'Local Richness, number of times 4 cell-types present',	'Local Richness, number of times 5 cell-types present']


features_csv['Local Shannon index skew'] = features_csv[shannon_group].skew(1)
features_csv['Local Simpson index skew'] = features_csv[simpson_group].skew(1)
features_csv['Local Renyi entropy (at alpha=infinity) skew'] = features_csv[renqi_group].skew(1)
features_csv['Local richness skew'] = features_csv[richness_group].skew(1)


features_csv = features_csv.drop(shannon_group, axis=1)
features_csv = features_csv.drop(simpson_group, axis=1)
features_csv = features_csv.drop(renqi_group, axis=1)
features_csv = features_csv.drop(richness_group, axis=1)

feats_size = features_csv.shape[1]


print('After removing local histogram bins', features_csv.shape)
print('Feat size after:', feats_size)


After removing local histogram bins (825, 351)
Feat size after: 351
